In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'mystic-hull-399003' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=mystic-hull-399003:US:bquxjob_74437eb0_18ab97d5e73)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_74437eb0_18ab97d5e73') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT
  stn,
  date,
  -- Use the IF function to replace 9999.9 values, which the dataset description explains is the default value when temperature is missing, with NULLs instead.
IF
  ( TEMP=9999.9, NULL, TEMP) AS temperature,
  -- Use the IF function to replace 999.9 values, which the dataset description explains is the default value when wind speed is missing, with NULLs instead.
IF
  ( wdsp="999.9", NULL, CAST(wdsp AS Float64)) AS wind_speed,
  -- Use the IF function to replace 99.99 values, which the dataset description explains is the default value when precipitation is missing, with NULLs instead.
IF
  ( prcp=99.99, 0, prcp) AS precipitation
FROM
  `bigquery-public-data.noaa_gsod.gsod2020`
WHERE
  stn="725030" -- La Guardia
  OR stn="744860" -- JFK
ORDER BY
  date DESC,
  stn ASC;


# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_74437eb0_18ab97d5e73') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,stn,date,temperature,wind_speed,precipitation
0,725030,2020-12-31,45.8,9.8,0.10
1,744860,2020-12-31,44.1,9.9,0.06
2,725030,2020-12-30,35.2,8.7,0.00
3,744860,2020-12-30,32.5,8.3,0.00
4,725030,2020-12-29,42.1,13.6,0.00
...,...,...,...,...,...
727,744860,2020-01-03,44.1,5.6,0.04
728,725030,2020-01-02,39.8,8.6,0.00
729,744860,2020-01-02,37.9,10.1,0.00
730,725030,2020-01-01,39.8,13.7,0.01


## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
results.describe()

,temperature,wind_speed,precipitation
count,732.000000,732.000000,732.000000
mean,57.169945,9.557240,0.116352
std,15.647524,3.666223,0.283791
min,21.200000,2.600000,0.000000
25%,44.300000,6.800000,0.000000
50%,55.050000,8.950000,0.000000
75%,71.525000,11.500000,0.070000
max,90.000000,22.100000,2.470000
